# graphblas.matrix_select

This example will go over how to compile MLIR code (using the GraphBLAS dialect) to a function callable from Python.

The example MLIR code we’ll use here will demonstrate how the `graphblas.matrix_select` op from the GraphBLAS dialect works. 

Let’s first import some necessary modules and generate an instance of our JIT engine.

In [1]:
import mlir_graphblas
import mlir_graphblas.sparse_utils
import numpy as np

engine = mlir_graphblas.MlirJitEngine()

Here are the passes we'll use. The pass `--graphblas-lower` is necessary to lower the GraphBLAS dialect.

In [2]:
passes = [
    "--graphblas-lower",
    "--sparsification",
    "--sparse-tensor-conversion",
    "--linalg-bufferize",
    "--func-bufferize",
    "--tensor-bufferize",
    "--tensor-constant-bufferize",
    "--finalizing-bufferize",
    "--convert-linalg-to-loops",
    "--convert-scf-to-std",
    "--convert-std-to-llvm",
]

Similar to our examples using `graphblas.convert_layout`, we'll need some helper functions to convert sparse tensors to dense tensors. 

In [3]:
mlir_text = """
#trait_densify_csr = {
  indexing_maps = [
    affine_map<(i,j) -> (i,j)>,
    affine_map<(i,j) -> (i,j)>
  ],
  iterator_types = ["parallel", "parallel"]
}

#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @csr_densify4x4(%argA: tensor<4x4xf64, #CSR64>) -> tensor<4x4xf64> {
  %output_storage = constant dense<0.0> : tensor<4x4xf64>
  %0 = linalg.generic #trait_densify_csr
    ins(%argA: tensor<4x4xf64, #CSR64>)
    outs(%output_storage: tensor<4x4xf64>) {
      ^bb(%A: f64, %x: f64):
        linalg.yield %A : f64
    } -> tensor<4x4xf64>
  return %0 : tensor<4x4xf64>
}
"""

Let's compile our MLIR code. 

In [4]:
engine.add(mlir_text, passes)

['csr_densify4x4']

## Overview of graphblas.matrix_select

Here, we'll show how to use the `graphblas.matrix_select` op. 

We'll give a brief explanation of how `graphblas.matrix_select` works here, but things will likely make more sense after reading the code examples shown later on. 

`graphblas.matrix_select` takes 1 sparse matrix operand in CSR format and returns several sparse matrices.
- Each result matrix has the same size as the given matrix.
- Each result matrix contains only a subset of the elements in the given sparse matrix
- Which elements are kept in each result matrix is determined by a given `selectors` attribute. 

Here's an example use of the `graphblas.matrix_select` op:
```
%answers = graphblas.matrix_select %sparse_tensor { selectors = ["triu", "tril"] } : tensor<?x?xf64, #CSR64> to tensor<?x?xf64, #CSR64>
```
- `%answers` contains 2 returned values. The first corresponds to "triu" and the second corresponds to "tril".
- The selector value of "triu" indicates that the result matrix will contain only the elements in the upper triangle of the given sparse matrix. This is similar to NumPy's [triu](https://numpy.org/doc/stable/reference/generated/numpy.triu.html). 
- The selector value of "tril" indicates that the result matrix will contain only the elements in the lower triangle of the given sparse matrix. This is similar to NumPy's [tril](https://numpy.org/doc/stable/reference/generated/numpy.tril.html). 

The supported options for the `selector` attribute are:
- `triu`: Select the elements in the upper triangle.
- `tril`: Select the elements in the lower triangle.
- `gt0`: Select the elements with values greater than 0.

Note that `graphblas.matrix_select` will fail if the given sparse matrix is not in CSR format. The result matrices will be in CSR format.

Let's create an example input CSR matrix.

In [5]:
indices = np.array(
    [
        [0, 3],
        [1, 3],
        [2, 0],
        [3, 0],
    ],
    dtype=np.uint64,
)
values = np.array([-1, 2, -3, 4], dtype=np.float64)
sizes = np.array([4, 4], dtype=np.uint64)
sparsity = np.array([False, True], dtype=np.bool8)

csr_matrix = mlir_graphblas.sparse_utils.MLIRSparseTensor(indices, values, sizes, sparsity)

In [6]:
dense_matrix = engine.csr_densify4x4(csr_matrix)

In [7]:
dense_matrix

array([[ 0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  2.],
       [-3.,  0.,  0.,  0.],
       [ 4.,  0.,  0.,  0.]])

## graphblas.matrix_select (Upper Triangle)

Let's select the upper triangle. 

In [8]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

module {
    func @select_triu(%sparse_tensor: tensor<?x?xf64, #CSR64>) -> tensor<?x?xf64, #CSR64> {
        %answer = graphblas.matrix_select %sparse_tensor { selectors = ["triu"] } : tensor<?x?xf64, #CSR64> to tensor<?x?xf64, #CSR64>
        return %answer : tensor<?x?xf64, #CSR64>
    }
}
"""

In [9]:
engine.add(mlir_text, passes)

['select_triu']

In [10]:
upper_triangle = engine.select_triu(csr_matrix)

In [11]:
engine.csr_densify4x4(upper_triangle)

array([[ 0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  2.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

The result looks sane. Let's verify that it has the same behavior as NumPy's [triu](https://numpy.org/doc/stable/reference/generated/numpy.triu.html).

In [12]:
np.all(np.triu(dense_matrix) == engine.csr_densify4x4(upper_triangle))

True

## graphblas.matrix_select (Lower Triangle)

Let's select the lower triangle. 

In [13]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

module {
    func @select_tril(%sparse_tensor: tensor<?x?xf64, #CSR64>) -> tensor<?x?xf64, #CSR64> {
        %answer = graphblas.matrix_select %sparse_tensor { selectors = ["tril"] } : tensor<?x?xf64, #CSR64> to tensor<?x?xf64, #CSR64>
        return %answer : tensor<?x?xf64, #CSR64>
    }
}
"""

In [14]:
engine.add(mlir_text, passes)

['select_tril']

In [15]:
lower_triangle = engine.select_tril(csr_matrix)

In [16]:
engine.csr_densify4x4(lower_triangle)

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-3.,  0.,  0.,  0.],
       [ 4.,  0.,  0.,  0.]])

The result looks sane. Let's verify that it has the same behavior as NumPy's [tril](https://numpy.org/doc/stable/reference/generated/numpy.tril.html).

In [17]:
np.all(np.tril(dense_matrix) == engine.csr_densify4x4(lower_triangle))

True

## graphblas.matrix_select (Greater Than Zero)

Let's select the elements whose values are greater than zero. 

Note that this is equivalent to passing the sparse matrix through a [ReLU activation function](https://en.wikipedia.org/wiki/Rectifier_%28neural_networks%29).

In [18]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

module {
    func @select_gt0(%sparse_tensor: tensor<?x?xf64, #CSR64>) -> tensor<?x?xf64, #CSR64> {
        %answer = graphblas.matrix_select %sparse_tensor { selectors = ["gt0"] } : tensor<?x?xf64, #CSR64> to tensor<?x?xf64, #CSR64>
        return %answer : tensor<?x?xf64, #CSR64>
    }
}
"""

In [19]:
engine.add(mlir_text, passes)

['select_gt0']

In [20]:
gt0_result = engine.select_gt0(csr_matrix)

In [21]:
engine.csr_densify4x4(gt0_result)

array([[0., 0., 0., 0.],
       [0., 0., 0., 2.],
       [0., 0., 0., 0.],
       [4., 0., 0., 0.]])

The result looks sane. Let's verify that it has the same behavior as NumPy.

In [22]:
np.all(dense_matrix*(dense_matrix>0) == engine.csr_densify4x4(gt0_result))

True

## graphblas.matrix_select (Multiple Selectors)

Let's use `graphblas.matrix_select` with multiple selectors. 

We'll write one function that'll use all three selectors shown above and will return 3 matrices. 

In [23]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

module {
    func @select_many(%sparse_tensor: tensor<?x?xf64, #CSR64>) -> (tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSR64>) {
        %triu, %tril, %gt0  = graphblas.matrix_select %sparse_tensor { selectors = ["triu", "tril", "gt0"] } : tensor<?x?xf64, #CSR64> to tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSR64>
        return %triu, %tril, %gt0 : tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSR64>, tensor<?x?xf64, #CSR64>
    }
}
"""

In [24]:
engine.add(mlir_text, passes)

['select_many']

In [25]:
triu_result, tril_result, gt0_result = engine.select_many(csr_matrix)

In [26]:
engine.csr_densify4x4(triu_result)

array([[ 0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  2.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [27]:
engine.csr_densify4x4(tril_result)

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-3.,  0.,  0.,  0.],
       [ 4.,  0.,  0.,  0.]])

In [28]:
engine.csr_densify4x4(gt0_result)

array([[0., 0., 0., 0.],
       [0., 0., 0., 2.],
       [0., 0., 0., 0.],
       [4., 0., 0., 0.]])

The result looks sane. Let's verify that they are consistent with NumPy.

In [29]:
np.all(np.triu(dense_matrix) == engine.csr_densify4x4(triu_result))

True

In [30]:
np.all(np.tril(dense_matrix) == engine.csr_densify4x4(tril_result))

True

In [31]:
np.all(dense_matrix*(dense_matrix>0) == engine.csr_densify4x4(gt0_result))

True